In [31]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import sys, os

In [32]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    elif t=='each':
        x=x.dropna().values
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [33]:
# RSI, %time above RSI
def RSI(series, period=14):
 #print(len(series))
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 #rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / pd.stats.moments.ewma(d, com=period-1, adjust=False)
 rs = pd.Series.ewm(u, com=period-1).mean()/pd.Series.ewm(d, com=period-1).mean()
 rsi = 100 - 100 / (1 + rs)
 return np.round(rsi.values[-1],2),get_percentile(rsi)#np.round(100*np.sum(np.where(rsi>rsi.values[-1],1,0))/len(rsi),2)

In [34]:
def consecutive(x,lag=4):
    c=0
    for i in x[-lag:][::-1]:
        if i>=0:
            c+=1
        else:
            return c
    return c    

def consecutive_increases(x):
    c=0
    latest=x.values[-1]
    for i in x[::-1]:
        if i==latest:
            continue
        if latest>i:
            c+=1
            latest=i
        else:
            return c
    return c    

In [35]:
# Consecutive increases, hist percentile, hist change mean 4 week, slope of 4 week macd
def macd_calcs(x):
    macd=x.ewm(span=12,min_periods=0,adjust=True,ignore_na=True).mean()-x.ewm(span=26,min_periods=0,adjust=True,ignore_na=True).mean()
    signal=macd.ewm(span=9,min_periods=0,adjust=True,ignore_na=True).mean()
    macd_hist=macd-signal
    return consecutive_increases(macd_hist), get_percentile(macd_hist),(macd_hist/macd_hist.shift(1)-1).rolling(4).mean().values[-1:][0],np.polyfit(x=[1,2,3,4],y=macd.values[-4:],deg=1,full=False)[0]

In [36]:
def get_intraday(ticker):
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': False}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['close'] is None)==False):
            return ([i['open'],i['high'],i['low'],i['close'],i['close']])
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': True}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['marketClose'] is None)==False):
            return([i['marketOpen'],i['marketHigh'],i['marketLow'],i['marketClose'],i['marketClose']])

In [45]:
holdings='NERD,Z,NTDOY,PBW,DECK,LAZY,BBY,ADBE,BLCN,GDXJ,AMZN,AAPL,FSLY,PTON,FUTU,BLCN,NOMD,GDXJ,BGS,AWK,SHW,EAT,CF,BATT,CAKE,TTEK,NWL,FAST,FXE,URA,ITB,GOLD,SPY,DOCU,ZM,PTON,MSFT,NVDA,EA,CRM,VEEV,ADBE,QQQ,TSLA,W,ESTC,GDX,CDNS,MELI,ARKF,TTD,WK,SNAP,ESPO,NEM,YETI,FCX'


In [46]:
holdings=holdings.split(',')

In [17]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
holdings=['SPY']

In [47]:
res=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

In [48]:
holdings

['NERD',
 'Z',
 'NTDOY',
 'PBW',
 'DECK',
 'LAZY',
 'BBY',
 'ADBE',
 'BLCN',
 'GDXJ',
 'AMZN',
 'AAPL',
 'FSLY',
 'PTON',
 'FUTU',
 'BLCN',
 'NOMD',
 'GDXJ',
 'BGS',
 'AWK',
 'SHW',
 'EAT',
 'CF',
 'BATT',
 'CAKE',
 'TTEK',
 'NWL',
 'FAST',
 'FXE',
 'URA',
 'ITB',
 'GOLD',
 'SPY',
 'DOCU',
 'ZM',
 'PTON',
 'MSFT',
 'NVDA',
 'EA',
 'CRM',
 'VEEV',
 'ADBE',
 'QQQ',
 'TSLA',
 'W',
 'ESTC',
 'GDX',
 'CDNS',
 'MELI',
 'ARKF',
 'TTD',
 'WK',
 'SNAP',
 'ESPO',
 'NEM',
 'YETI',
 'FCX']

In [49]:
c=1
for ticker in holdings:
    print(ticker)
    time.sleep(1)
    c+=1
    if c==4:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey=YEUC2DL0F7KC43PB&outputsize=full'.format(ticker)
    response = requests.get(
        url
#        params=d,
        )
    #print(response.text)
    #print(url)
    try:
        spy=[[],[],[],[],[],[],[]]
        #print(response.text)
        for key, val in response.json()['Time Series (Daily)'].items():
            spy[0].append(float(val['1. open']))
            spy[1].append(float(val['2. high']))
            spy[2].append(float(val['3. low']))
            spy[3].append(float(val['4. close']))
            spy[4].append(float(val['5. adjusted close']))
            #spy[5].append(float(val['6. volume']))
            #spy[6].append(float(val['7. dividend amount']))
        t=list(response.json()['Time Series (Daily)'].keys())
        
        try:
            if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
                x=get_intraday(ticker)
                spy[0].insert(0,float(x[0]))
                spy[1].insert(0,float(x[1]))
                spy[2].insert(0,float(x[2]))
                spy[3].insert(0,float(x[3]))
                spy[4].insert(0,float(x[4]))
                t.insert(0,str(pd.Timestamp.today().date()))

                #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
        except:
            print(ticker,'IEX')

        


        spy=pd.DataFrame({'Date':t,
                     'open':spy[0],
                      'high':spy[1],
                      'low':spy[2],
                      'close':spy[3],
                      'adj_close':spy[4]#,
                    #  'volume':spy[5],
                    #  'dividend':spy[6],
                     })
        spy=spy.astype({'Date': 'datetime64'})
        spy=spy.sort_values(by='Date')
        spy['highest_high']=spy.high.rolling(22).max()
        spy['lowest_low']=spy.low.rolling(22).min()
        spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
        spy['k']=spy.k.rolling(3).mean()
        spy['k_change'] = spy.k/spy.k.shift(3)-1
        res[0].append(ticker)
        res[1].append(spy.iloc[-1]['Date']) 
        res[2].append(spy.iloc[-1]['k']) 
        res[3].append(spy.iloc[-1]['k_change'])
        res[4].append(np.nanpercentile(spy.k_change,q=10))
        res[5].append(get_percentile(spy.k_change))
        temp_res=RSI(spy.adj_close)
        res[6].append(temp_res[0])
        res[7].append(temp_res[1])
        temp_res=macd_calcs(spy.adj_close)
        res[8].append(temp_res[0])
        res[9].append(temp_res[1])
        res[10].append(spy.adj_close.rolling(90).std().values[-1])
        res[11].append((spy.adj_close/spy.adj_close.shift(5)-1).rolling(8).mean().values[-1])
        res[12].append((spy.adj_close/spy.adj_close.shift(22)-1).values[-1])
        res[13].append((spy.adj_close/spy.adj_close.shift(5)-1).values[-1])
        res[14].append((spy.adj_close/spy.adj_close.shift(1)-1).values[-1])
        time.sleep(1)
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(ticker,exc_type, exc_tb.tb_lineno)  


NERD
NERD IEX
Z
NTDOY
NTDOY IEX
PBW
PBW IEX
DECK
LAZY
BBY
ADBE
BLCN
BLCN IEX
GDXJ
AMZN
AAPL
FSLY
PTON
FUTU
BLCN
BLCN IEX
NOMD
GDXJ
BGS
BGS IEX
AWK
SHW
EAT
CF
BATT
BATT IEX
CAKE
TTEK
NWL
FAST
FXE
FXE IEX
URA
URA IEX
ITB
GOLD
SPY
DOCU
ZM
PTON
MSFT
NVDA
EA
CRM
VEEV
ADBE
QQQ
TSLA
W
ESTC
GDX
CDNS
MELI
ARKF
ARKF IEX
TTD
WK
SNAP
ESPO
ESPO IEX
NEM
YETI
FCX


In [42]:
spy.to_csv('spy.csv')

In [43]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_10_percentile':res[4],
             'k_change_percentile':res[5],
             'RSI':res[6],
             'RSI_percentile':res[7],
             'hist_cons':res[8],    
             'hist_percentile':res[9],
             '90dayStd':res[10],
             'ret1wk_mean':res[11],
             '22dayret':res[12],
             '5dayret':res[13],
             '1dayret':res[14]         
             }
            )

In [44]:
res.sort_values(by='hist_cons',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret,5dayret,1dayret
0,LAZY,2020-08-25,0.240381,-0.639467,-0.426466,3.05,49.15,56.48,0,0.00,3.913021,-0.041489,0.048560,-0.150667,0.061667
23,FXE,2020-08-24,0.608063,-0.308955,-0.538870,20.60,55.77,66.90,0,18.32,3.269137,0.003076,0.016188,-0.006794,-0.000180
24,URA,2020-08-24,0.526372,-0.084270,-0.574760,41.19,54.91,71.74,0,43.55,0.580853,0.009909,-0.019835,-0.006700,0.008503
25,ITB,2020-08-25,0.924531,0.034307,-0.485085,56.07,71.19,95.68,0,46.89,6.778321,0.029282,0.123899,0.009710,-0.010921
28,DOCU,2020-08-25,0.439178,-0.210397,-0.374222,20.32,50.65,34.55,0,4.94,39.350987,0.034524,0.066300,-0.027975,-0.004303
30,PTON,2020-08-25,0.535501,-0.198374,-0.358702,22.93,50.50,27.19,0,16.02,12.366768,0.016403,0.065675,-0.056431,-0.014805
32,NVDA,2020-08-25,0.935339,0.058302,-0.466101,58.77,74.92,95.50,0,99.35,59.885667,0.068807,0.242386,0.033012,-0.004304
33,EA,2020-08-25,0.598954,-0.116059,-0.465418,34.15,53.55,54.35,0,11.26,10.624128,0.007845,0.047471,-0.006453,-0.004707
35,VEEV,2020-08-25,0.578935,-0.268519,-0.390558,16.93,52.21,42.67,0,4.23,29.060469,0.016698,0.036495,-0.020792,0.005796
38,TSLA,2020-08-25,0.868068,-0.077268,-0.449200,36.89,72.79,91.75,0,99.73,382.141146,0.204960,0.403677,0.054009,-0.012506


In [16]:
res.sort_values(by='ret1wk_mean',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret,5dayret,1dayret
7,FSLY,2020-08-19,0.137821,0.276792,-0.522081,73.29,46.84,33.55,3,4.72,26.626991,-0.125753,-0.066529,0.042857,0.010397
9,FUTU,2020-08-19,0.520341,-0.020655,-0.480052,46.92,50.78,47.88,0,0.00,9.052474,-0.057066,0.128843,-0.080990,-0.076836
28,DOCU,2020-08-19,0.523533,0.835185,-0.374287,92.82,57.57,52.90,5,2.92,40.291654,-0.037054,0.043467,0.090378,0.006206
8,PTON,2020-08-19,0.636226,0.449898,-0.360970,86.57,58.14,53.99,3,19.82,12.555127,-0.030160,0.066141,0.071939,-0.011888
30,PTON,2020-08-19,0.636226,0.449898,-0.360970,86.57,58.14,53.99,3,19.82,12.555127,-0.030160,0.066141,0.071939,-0.011888
50,NEM,2020-08-19,0.508739,0.790829,-0.488904,87.71,51.47,52.15,3,5.96,4.044548,-0.029813,0.031002,0.050643,-0.022452
42,GDX,2020-08-19,0.523402,0.644829,-0.526798,83.26,52.55,57.46,5,11.26,3.910046,-0.026779,0.031575,0.061815,-0.023558
4,GDXJ,2020-08-19,0.512418,0.299931,-0.537336,73.11,52.09,59.03,0,9.85,7.532279,-0.026278,0.040106,0.054451,-0.030308
12,GDXJ,2020-08-19,0.512418,0.299931,-0.537336,73.11,52.09,59.03,0,9.85,7.532279,-0.026278,0.040106,0.054451,-0.030308
41,CLX,2020-08-19,0.334893,0.692165,-0.412416,89.80,51.56,43.41,3,0.46,14.978820,-0.025192,0.007835,0.006455,0.001074


In [402]:
res[(res.RSI_percentile<93) & (res.ret1wk_mean>=0.023557) & (res['1dayret']>=-0.02)].sort_values(by='ret1wk_mean',ascending=False)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret,5dayret,1dayret
4,AAPL,2020-08-12,0.894281,-0.034683,-0.426907,42.71,70.15,86.06,0,99.71,50.720804,0.097549,0.186981,0.029687,0.034251
7,FUTU,2020-08-12,0.556807,-0.305225,-0.483387,17.26,58.93,70.98,0,6.63,8.897628,0.077063,0.153833,-0.141067,0.018088
44,TTD,2020-08-12,0.514921,-0.405681,-0.379633,9.12,55.39,48.14,0,15.71,84.549160,0.057320,0.078339,-0.063761,0.041995
1,BLCN,2020-08-11,0.808652,-0.147628,-0.434118,23.99,67.54,86.41,0,74.19,3.767007,0.051899,0.096313,0.007890,-0.002603
8,BLCN,2020-08-11,0.808652,-0.147628,-0.434118,23.99,67.54,86.41,0,74.19,3.767007,0.051899,0.096313,0.007890,-0.002603
50,FCX,2020-08-12,0.733621,-0.059941,-0.495752,40.02,60.54,75.17,0,44.81,2.146948,0.048024,0.081029,0.007516,0.008238
30,NVDA,2020-08-12,0.794434,-0.132518,-0.466858,30.21,66.58,84.56,1,92.75,54.710242,0.043819,0.137432,0.013024,0.053802
42,MELI,2020-08-12,0.523283,-0.418773,-0.445485,11.17,56.05,55.90,0,3.57,190.512671,0.040217,0.136277,-0.089485,0.014345
11,BGS,2020-08-12,0.735897,-0.114695,-0.465117,32.63,66.46,87.70,0,67.93,3.549423,0.038126,0.193964,-0.022083,0.029851
43,ARKF,2020-08-12,0.697941,-0.258489,-0.318967,12.53,60.94,50.94,0,16.36,4.979039,0.037225,0.092751,-0.034631,0.013306


In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1
    https://cloud.iexapis.com/stable/stock/CWH/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=30&chartIEXWhenNull=True

In [ ]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [ ]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [ ]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

In [ ]:
date=list(response.json()['Time Series (Daily)'].keys())

In [ ]:
date.insert(0,response2.json()[0]['date'])

In [ ]:
date

In [ ]:
spy


In [ ]:
np.nanpercentile(spy.k_change,q=10)

In [ ]:
response.json()[0]

In [ ]:
pd.to_datetime('today')

In [ ]:
pd.Timestamp.today().date()

In [ ]:
pd.to_datetime(t[0]).date()

In [ ]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

In [ ]:
spy.adj_close.rolling(90).std().values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(22)-1).values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1]

In [21]:
spy=spy.set_index('Date')#['2020-07-01':]

In [22]:
spy['ret']=spy.adj_close/spy.adj_close.shift(5)-1

In [23]:
#spy['ret_3']=spy.ret.rolling(3).mean()

In [24]:
np.nanpercentile(spy.k_change,q=10),np.nanpercentile(spy.ret,q=10)

(-0.45135471656489845, -0.02648284785577889)

In [25]:
def each_percentile(x,val):
    return (x<val).sum()/len(x)

In [26]:
spy['ret_p']=spy.ret.apply(lambda x: each_percentile(spy.ret,x))

In [27]:
#spy['ret_3_p']=spy.ret_3.apply(lambda x: each_percentile(spy.ret,x))

In [29]:
spy['2020-02-01':'2020-02-28']

,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change,ret,ret_p
Date,,,,,,,,,,,
2020-02-03,323.3500,326.1600,323.2200,324.12,320.7214,332.95,320.36,0.336789,-0.219039,0.001917,0.465152
2020-02-04,328.0700,330.0100,327.7200,329.06,325.6096,332.95,320.36,0.371037,-0.313026,0.006638,0.577812
2020-02-05,332.2701,333.0900,330.6700,332.86,329.3697,333.09,320.36,0.657202,0.619805,0.019105,0.823181
2020-02-06,333.9100,334.1900,332.8000,333.98,330.4780,334.19,320.73,0.885785,1.630085,0.019226,0.825664
2020-02-07,332.8200,333.9941,331.6000,332.20,328.7166,334.19,320.73,0.939495,1.532077,0.032543,0.933741
2020-02-10,331.2300,334.7500,331.1900,334.68,331.1706,334.75,320.73,0.943853,0.436169,0.032580,0.934123
2020-02-11,336.1697,337.0200,334.6840,335.26,331.7445,337.02,320.73,0.913040,0.030769,0.018841,0.820126
2020-02-12,336.8300,337.6500,336.4300,337.42,333.8819,337.65,320.73,0.957791,0.019474,0.013699,0.731144
2020-02-13,335.8621,338.1200,335.5600,337.06,333.5257,338.12,320.73,0.939137,-0.004997,0.009222,0.639870


In [ ]:
spy['2020-02-20':'2020-3-31']


So something can go down at random 
look to see how other stocks have done. if it is aleader like CWH or MELI NEED TO track and buy
that is part of the in-cycle market leader

In [50]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [32]:
fetch_these=['Earnings','Market Cap','P/E','Forward P/E','Sales','Income','Dividend %','Short Float', 'Target Price'
             ,'ROE','Perf Week','Perf Month','Perf Quarter','Perf Year','Insider Own','Insider Trans']

In [33]:

def fetcher(soup, fetch_these,ticker):
    temp=[]
    temp.append(ticker)
    temp.append(pd.datetime.now())
    for i in fetch_these:
        x=soup.find('table', class_='snapshot-table2').find('td', string=i).find_next('td').text
        temp.append(x)
    return temp

In [34]:
stock_stats=[]
for j in holdings:
    try:
        url=str.format('https://finviz.com/quote.ashx?t={}',j)
        r=requests.get(url)
        soup=BeautifulSoup(r.text)
        stock_stats.append(fetcher(soup,fetch_these,j))
        time.sleep(np.random.randint(1,5))
        if np.random.uniform()<=0.3:
            time.sleep(4)
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(j, exc_type, exc_tb.tb_lineno)

W <class 'AttributeError'> 7


/Users/rittikghosh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [35]:
stock_stats

[]

In [39]:
j='W'
url=str.format('https://finance.yahoo.com/quote/W?p=W&.tsrc=fin-srch')
r=requests.get(url)

In [41]:
soup=BeautifulSoup(r.text)

In [ ]:
https://cloud.iexapis.com/stable/stock/W/upcoming-earnings?token=pk_9469e57bc642462d905e4dc0003523ab
    http://q4finance.com/analysis/earning%20Drift/BA

In [42]:
url='http://q4finance.com/analysis/earning%20Drift/BA'
r=requests.get(url)

In [44]:
soup=BeautifulSoup(r.text)

In [46]:
soup.find('table', class_='table table-responsive img-fluid  table_text_nowrap').text

AttributeError: 'NoneType' object has no attribute 'text'

In [51]:
url='https://service.q4finance.com/analysis/etf_latest_price'
r=requests.post(url, data={'identifier':'SPY'})

In [ ]:
fields=['ticker','fund_aum','asset_class','focus','development_level','region','is_levered','options_available']

In [57]:
json.loads(r.text)['static_data']#['fund_aum']

{'date': '2018-09-24',
 'ticker': 'SPY',
 'issuer': 'SSgA',
 'description': 'SPDR S&P 500 ETF Trust',
 'inception_date': '1993-01-22',
 'primary_benchmark': 'S&P 500 TR USD',
 'tax_classification': 'Regulated Investment Company',
 'is_etn': 0.0,
 'fund_aum': 280305259398.0,
 'avg_daily_trading_volume': 63253888.0,
 'asset_class': 'Equity',
 'category': 'Size and Style',
 'focus': 'Large Cap',
 'development_level': 'Developed Markets',
 'region': 'North America',
 'is_levered': 0.0,
 'leverage_factor': None,
 'active': 0.0,
 'administrator': 'State Street Bank and Trust Company',
 'advisor': None,
 'custodian': None,
 'distributor': 'ALPS Distributors, Inc.',
 'portfolio_manager': None,
 'subadvisor': None,
 'transfer_agent': 'State Street Bank and Trust Company',
 'trustee': 'State Street Global Advisors Trust Company',
 'futures_commission_merchant': None,
 'fye': 'September 30',
 'dist_freq': 'Q',
 'listing_exchange': 'NYSE Arca, Inc.',
 'creation_unit_size': 50000.0,
 'creation_fee'

In [52]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

KeyError: 'data'

In [94]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2019-07-30
2019-04-30
2019-01-29
2018-11-01
2018-07-31
2018-05-01


In [261]:
df=pd.DataFrame()
tickers=pd.read_csv('stock_names.csv')['ticker']
#tickers=['MSFT','AAPL','BLK','HD']

In [262]:
c=0
for ticker in tickers:
    c+=1
    url='https://service.q4finance.com/analysis/historic_return_graph'
    r=requests.post(url, data={'identifier':ticker})
    peer_of=[]
    peers=[]
    for s in json.loads(r.text)['response_data']:
        if s['identifier'] == ticker:
            continue
        peers.append(s['identifier'])
        peer_of.append(ticker)
    df=df.append(pd.DataFrame(zip(peer_of,peers)),ignore_index=True)
    if c==3:
        time.sleep(2)
        c=0

In [266]:
len(df.iloc[:,0].unique())

2382

In [267]:
df.to_csv('company_peers.csv')

In [63]:
tickers=['SPY','QQQ']

In [78]:
tickers=pd.read_csv('etf_names.csv')['ticker']

In [79]:
fields=['ticker','fund_aum','asset_class','focus','development_level','region','is_levered','options_available']

In [81]:
store=[[],[],[],[],[],[],[],[]]
c=0
for t in tickers:
    try:
        c+=1
        url='https://service.q4finance.com/analysis/etf_latest_price'
        r=requests.post(url, data={'identifier':t})
        for idx,f in enumerate(fields):
            store[idx].append(json.loads(r.text)['static_data'][f])
        if c==15:
            time.sleep(2)
            c=0
    except:
        print(t)

PZI
AUSE
WMCR
PUW
EQWS
PAF
ENY
HGI
AGF
BZF
PSAU
PAGG
PXR
VXX
VXZ
PRB
YAO
PLND
QQQC
WREI
DLBL
ALD
FWDD
FWDI
DSUM
TTFS
BSCI
BSJI
CMDT
DBAP
DXJF
IBDH
LALT
JHDG
DHVW
OASI
HAHA
EDOM
GSD
HGSD
UDBI
DDBI
EDBI
DDEZ
DDJP
GNRX
EVSTC
EVLMC
EVGBC
JPIH
JPEH
BUZ
MPCT
VMIN
OVLC
OEW
DWLV
PMPT
SHNY
YESR
KOR
IVFGC
IVFVC
IVENC
OILB
MOGLC
ARVR
SWIN
CUMB
QXMI
QXRR
FOANC
GAZB
ONTL
BEF
SPVM
ICAN
GMFL
VESH
FANZ
EMBU
FMDG
LOGO
OKDCC
EVFTC
HFGIC
JBRI
CRUSC
JJAB
JJUB
BJO
JJCB
BALB
JJEB
JJGB
JJMB
COWB
BJJN
PGMB
JJPB
JJSB
SGGB
JJTB
BVNSC
PPTB
RPIBC
BKC


In [82]:
data=pd.DataFrame({
    'ticker':store[0]
    ,'fund_aum':store[1]
    ,'asset_class':store[2]
    ,'focus':store[3]
    ,'development_level':store[4]
    ,'region':store[5]
    ,'is_levered':store[6]
    ,'options_available':store[7]
})

In [83]:
data.to_csv('etf_data.csv')

In [84]:
data

,ticker,fund_aum,asset_class,focus,development_level,region,is_levered,options_available
0,FLGE,2.409798e+09,Equity,Large Cap,Developed Markets,North America,1.0,0.0
1,RDVY,5.083036e+08,Equity,High Dividend Yield,Developed Markets,North America,0.0,0.0
2,KBA,3.215242e+08,Equity,Broad Equity,Emerging Markets,Asia-Pacific,0.0,1.0
3,IPOS,2.248622e+06,Equity,Alpha-Seeking,Developed Markets,Global Ex-U.S.,0.0,0.0
4,CEFL,2.305220e+08,Multi Asset,Target Outcome,Blended Development,Global,1.0,0.0
...,...,...,...,...,...,...,...,...
2053,IUS,0.000000e+00,Equity,Large Cap,Developed Markets,North America,0.0,0.0
2054,IUSS,0.000000e+00,Equity,Small Cap,Developed Markets,North America,0.0,0.0
2055,ISDX,0.000000e+00,Equity,Large Cap,Developed Markets,Developed Markets,0.0,0.0
2056,ISDS,0.000000e+00,Equity,Small Cap,Developed Markets,Developed Markets,0.0,0.0


In [140]:
for f in fields:
    print(json.loads(r.text)['static_data'][f])

AAPL
Tim Cook
www.apple.com
Consumer Durables
Electronic Equipment
https://www.sec.gov/cgi-bin/browse-edgar?CIK=AAPL&owner=exclude&action=getcompany


In [130]:
store=[[],[]]
for etf in json.loads(r.text)['data']:
    store[0].append(etf['identifier'])
    store[1].append(etf['name'])

In [132]:
pd.DataFrame(zip(store[0],store[1]),columns=['ticker','name']).to_csv('stock_names.csv')

In [368]:
import this #Zen of python

In [364]:
import antigravity # Xkcd

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-365-6d5c5b2f0daf>, line 4)

In [369]:
import this

In [370]:
from __future__ import braces

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-370-6d5c5b2f0daf>, line 4)

so have a list of know winners, YETI, W, MELI
so when to buy and when to sell 
what to replace, needs quantifications 

conversion 
and positive intent status 
ple added payment post perlin
no hits by ineeligiblity